# Exercise 1: SLR Pulse Design


Welcome! This first exercise will cover the design of SLR pulses for MRI using Python. 

Documentation for all pulse design tools can be found [here](https://sigpy.readthedocs.io/en/latest/mri_rf.html).

Prior to beginning this exercise (and at the beginning of all additional exercise notebooks) we will need to import the packages we will use:


In [ ]:
# typical sigpy and numpy imports
import numpy as np
import sigpy as sp
import sigpy.mri.rf as rf # import for our RF pulse design tools 
import sigpy.plot as pl

import matplotlib.pyplot as pyplot
pyplot.rcParams.update({'figure.max_open_warning': 0})  # We'll be showing many figures

## Problem 1a: design an inversion pulse

Large-tip inversion pulses cannot be effectively designed by the small-tip-angle approximation alone, and should instead be designed by iterative methods or the SLR method, which we will use here.

In SigPy, basic SLR pulses are designed using the *sigpy.mri.rf.slr.dzrf* function, modeled after John Pauly's SLR design tools. Documentation for dzrf() is [here](https://sigpy.readthedocs.io/en/latest/generated/sigpy.mri.rf.slr.dzrf.html#sigpy.mri.rf.slr.dzrf).

For this example, we want to design an inversion pulse that will provide spatial saturation with a high degree of signal dephasing.

* Using dzrf(), design an inversion pulse lasting 0.5 ms.
* Assume your hardware dwell time is 4e-6s. 
* To start, give your pulse 8 zero crossings. 
* Use a filter type that concentrates the pulse energy at the front of the pulse. This will result in signal dephasing.

In [ ]:
N = 128
tb = 8
d1 = 0.01
d2 = 0.01
ptype = 'inv'
ftype = 'max'

In [ ]:
pulse = rf.slr.dzrf(N, tb, ptype, ftype, d1, d2)
pl.LinePlot(pulse, mode='r', title = 'Real Component')

## Problem 1b: simulate the inversion pulse's profile

* Simulate the Mz profile created by this pulse.
* This is done in 2 steps: first, find get the a and b coefficients that correspond to the RF pulse. Second, 

*Hint:* the inversion profile is related to the 

In [ ]:
[a, b] = rf.sim.abrm(pulse, np.arange(-2*tb, 2*tb, 0.01), True)
Mz = 1-2*np.abs(b)**2


In [ ]:
pl.LinePlot(Mz, mode='r')

# Problem 1c: alter the time-bandwidth product

* Now, go back and rerun the code above, but change the time-bandwidth product to 2. Note the change in the inversion profile!

# Problem 1d: root-flip the inversion pulse

For the final problem of exercise 1, we will explore the rf.mri.slr.dzrf() source code, and design a root-flipped SLR inversion pulse using what we find there, which we can compare to the standard SLR inversion pulse designed in 1a.

Root flipping can be performed using the rf.slr.root_flip() function. This function is used in the following fashion:

[pulse, bRootFlipped] = rf.slr.root_flip(b, d1, flip, tb)

To design the root flipped pulse, we will need b (the SLR beta parameter), d1 (the passband ripple level), flip (the flip angle of our pulse in radians) and tb (our time bandwidth product). All of these are knowns, aside from b.

**Your assignment is to design b by extracting 4 lines of code from the [rf.mri.slr.dzrf() source code](https://sigpy.readthedocs.io/en/latest/_modules/sigpy/mri/rf/slr.html#dzrf).**

*Hint #1: you will want 1 line of code that calculates ripples from d1 and d2, 2 lines of code that design a minphase b.*

*Hint #2: you will want to end with a 4th line of code that multiplies bsf by b.*


In [ ]:
flip = np.pi # 180 degree inversion
d1 = 0.01
d2 = 0.001
tb = 8

In [ ]:
# root-flipped pulse

[bsf, d1, d2] = rf.slr.calc_ripples(ptype, d1, d2)
b = rf.slr.dzmp(N, tb, d1, d2)
b = b[::-1]
b = bsf*b

[pulse, bRootFlipped] = rf.slr.root_flip(b, d1, flip, tb)
pl.LinePlot(pulse)

In [ ]:
[a, b] = rf.sim.abrm(pulse, np.arange(-2*tb, 2*tb, 0.01), True)
Mz = 1-2*np.abs(b)**2

In [ ]:
pl.LinePlot(Mz, mode='r')